In [ ]:
import os
import nest_asyncio
from scrapy.crawler import CrawlerRunner
from twisted.internet import reactor
from scrapy import Spider

# 修復 Jupyter 與 Scrapy 的事件循環問題
nest_asyncio.apply()

# 設置工作目錄
os.chdir(r"C:\Users\USER\Downloads\NLP-Courses\Scrapy")
print("當前工作目錄:", os.getcwd())

# 定義爬蟲類
class BranchesSpider(Spider):
    name = "branches"
    allowed_domains = ["service.standardchartered.com.tw"]
    start_urls = [
        "https://service.standardchartered.com.tw/location_finder/branch_table/Index"
    ]

    def parse(self, response):
        rows = response.xpath('//table[contains(@class, "branch_table")]/tbody/tr')
        for row in rows:
            yield {
                "branch_name": row.xpath('td[1]/text()').get(default="").strip(),
                "address": row.xpath('td[2]/text()').get(default="").strip(),
                "phone": row.xpath('td[3]/text()').get(default="").strip(),
            }

# 初始化爬蟲執行器
runner = CrawlerRunner({
    'FEED_FORMAT': 'json',  # 設置輸出格式
    'FEED_URI': 'branches.json',  # 輸出文件路徑
})

# 啟動爬蟲
d = runner.crawl(BranchesSpider)
d.addBoth(lambda _: reactor.stop())

# 啟動事件循環
reactor.run()

print("爬取完成！請檢查 branches.json 文件。")
